# Elementy Inteligencji Obliczeniowej - Sieci Neuronowe


---

**Prowadzący:** Jakub Bednarek<br>
**Kontakt:** jakub.bednarek@put.poznan.pl<br>
**Materiały:** [Strona WWW](http://jakub.bednarek.pracownik.put.poznan.pl)

---

## Uwaga

* **Aby wykonać polecenia należy najpierw przejść do trybu 'playground'. File -> Open in Playground Mode**
* Nowe funkcje Colab pozwalają na autouzupełnianie oraz czytanie dokumentacji

## Cel ćwiczeń:
- zapoznanie się z Keras subclassing API
- stworzenie własnych modeli i warstw z wykorzystaniem Keras subclassing API
- wykorzystanie podstawowych mechanizmów regularyzacji: Dropout i Batch normalization

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
import numpy as np

TensorFlow 2.x selected.


In [0]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Layer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, RMSprop
from tensorflow.python.keras import backend as K


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train = x_train[:, :, :, np.newaxis].astype('float32')
x_test = x_test[:, :, :, np.newaxis].astype('float32')
x_train /= 255
x_test /= 255

# x_train = x_train.reshape(60000, 784)
# x_test = x_test.reshape(10000, 784)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## Tworzenie własnych modeli i warstw 
https://www.tensorflow.org/guide/keras/custom_layers_and_models

Przykładowy model z warstwami gęstymi dla danych MNIST:

In [0]:
class DenseModel(Model):

  def __init__(self, num_classes=10):
    super(DenseModel, self).__init__(name='my_model')
    self.num_classes = num_classes
    # Define your layers here.
    self.dense_1 = Dense(512, input_shape=(784,), activation='relu')
    self.dense_2 = Dense(512, activation='relu')
    self.dense_3 = Dense(num_classes, activation='softmax')

  def call(self, inputs):
    # Define your forward pass here,
    # using layers you previously defined (in `__init__`).
    x = self.dense_1(inputs)
    x = self.dense_2(x)
    return self.dense_3(x)

model = DenseModel(num_classes=10)

In [8]:
model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=3)

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 19s 317us/sample - loss: 0.1919 - accuracy: 0.9435
Epoch 2/3
60000/60000 [==============================] - 18s 306us/sample - loss: 0.1024 - accuracy: 0.9733
Epoch 3/3
60000/60000 [==============================] - 18s 305us/sample - loss: 0.0849 - accuracy: 0.9803


Przykład własnej warstwy.

In [9]:
class CustomLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(CustomLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                      shape=(int(input_shape[1]), self.output_dim),
                                      initializer='uniform',
                                      trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)


class CustomModel(Model):

    def __init__(self, num_classes=10):
        super(CustomModel, self).__init__(name='custom_model')
        self.num_classes = num_classes
        # Define your layers here.
        self.dense_1 = CustomLayer(output_dim=512, input_shape=(784,))
        self.dense_2 = CustomLayer(output_dim=512, input_shape=(512,))
        self.dense_3 = CustomLayer(output_dim=num_classes, input_shape=(512,))

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.dense_1(inputs)
        x = K.relu(x)
        x = self.dense_2(x)
        x = K.relu(x)
        x = self.dense_3(x)
        x = K.sigmoid(x)
        return x

model = CustomModel(num_classes=10)

model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.fit(x_train, y_train, batch_size=128, epochs=3)




Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 8s 129us/sample - loss: 0.2862 - accuracy: 0.9115
Epoch 2/3
60000/60000 [==============================] - 7s 122us/sample - loss: 0.0969 - accuracy: 0.9705
Epoch 3/3
60000/60000 [==============================] - 7s 122us/sample - loss: 0.0626 - accuracy: 0.9804


### Zadanie 1
Na podstawie powyższego przykładu stwórz własny model kolejno składający się z:
- warstwy konwolucyjnej (Conv2D): 32 filtry 3x3,
- konwolucyjnej: 64 filtry 3x3,
- warstwy MaxPooling (MaxPooling2D): 2x2
- warstwy ukrytej gęstej (Dense): 128 neuronów,
- warstwy wyjściowej.

Ważne:
- w każdej warstwie poza warstwą wyjściową funkcją aktywacji powinno być relu,
- funkcja aktywacji dla warstwy wyjściowej to softmax,
- między częścią konwolucyjną a gęstą trzeba spłaszczyć tensor przy pomocy warstwy (Flatten),
- w przykładzie jest wykorzystywana sieć gęsta (dane są spłaszczone), sieci z warstwami konwolucyjnymi muszą otrzymać tensor 4-wymiarowy, zakomentuj linie "spłaszczające" podczas wczytywania danych.
```
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
```

In [36]:
class MyModel(Model):

    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # Define your layers here.
        self.conv_1 = Conv2D(32, (3,3), (1, 1),  activation='relu')
        self.conv_2 = Conv2D(64, (3,3), (1, 1),  activation='relu')
        self.maxPooling = MaxPooling2D((2, 2))
        self.flatten = Flatten()
        self.dense =Dense(128, 'relu')
        self.dense_2 = Dense(num_classes,'softmax')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.conv_1(inputs)
        x = self.conv_2(x)
        
        x = self.maxPooling(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.dense_2(x)
        return x

model = MyModel(num_classes=10)

model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



result1 = model.fit(x_train, y_train, batch_size=32, epochs=3,validation_data=(x_test,y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 138s 2ms/sample - loss: 0.1527 - accuracy: 0.9524 - val_loss: 0.0521 - val_accuracy: 0.9839
Epoch 2/3
60000/60000 [==============================] - 137s 2ms/sample - loss: 0.0416 - accuracy: 0.9876 - val_loss: 0.0345 - val_accuracy: 0.9892
Epoch 3/3
60000/60000 [==============================] - 137s 2ms/sample - loss: 0.0261 - accuracy: 0.9917 - val_loss: 0.0318 - val_accuracy: 0.9901


### Zadanie 2 
Na podstawie powyższego przykładu stwórz model bloku ResNet:
- w warstwach konwolucyjnych wykorzystaj padding='same', aby rozmiary tensorów się nie zmieniały,
- tego modelu nie trzeba budować i uczyć, zostanie on wykorzystany w kolejnym zadaniu,
- nie sugeruj się przykładem z własną warstwą, ten przykład jest podany tylko w celu zapoznania się z taką możliwością w TensorFlow, wszystkie zadania rozwiązujemy, tworząc własne modele.

![resnet](https://miro.medium.com/max/1000/1*6HDuqhUzP92iXhHoS0Wl3w.png)

In [0]:
class ResNet(Model):

    def __init__(self):
        super(ResNet, self).__init__(name='resnet')
        # Define your layers here.
        self.conv_1 = Conv2D(filters=32, kernel_size= (3,3), padding='same',activation='relu')
        self.conv_2 = Conv2D(filters=64, kernel_size = (3,3),  padding='same')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.conv_1(inputs)
        x = self.conv_2(x) + inputs
        x = K.relu(x)
        
        return x


### Zadanie 3
Zmodyfikuj model z zadania 1, zamieniając warstwy konwolucyjne na dwa modele bloku ResNet z zadania 2.

In [13]:
class SecondModel(Model):

    def __init__(self, num_classes=10):
        super(SecondModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # Define your layers here.
        self.resNet_1 = ResNet()
        self.resNet_2 = ResNet()
        self.maxPooling = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense =Dense(128, 'relu')
        self.dense_2 = Dense(num_classes,'softmax')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.resNet_1(inputs)
        x = self.resNet_2(x)
        x = self.maxPooling(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.dense_2(x)
        return x

model = SecondModel(num_classes=10)

model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



model.fit(x_train, y_train, batch_size=32, epochs=3,validation_data=(x_test,y_test))

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 480s 8ms/sample - loss: 0.1848 - accuracy: 0.9491
Epoch 2/3
60000/60000 [==============================] - 480s 8ms/sample - loss: 0.0388 - accuracy: 0.9883
Epoch 3/3
60000/60000 [==============================] - 480s 8ms/sample - loss: 0.0230 - accuracy: 0.9929


### Zadanie 4
Wykorzystując Keras Subclassing API, napisz Autoenkoder dla zbioru danych MNIST.
- stwórz osobny model Enkodera,
- stwórz osobny model Dekodera,
- połącz oba modele celem zbudowania Autoenkodera,
- można korzystać z warstw gęstych, nie trzeba korzystać z konwolucji,
- poprzednie zadania były przykładem klasyfikacji, w których wykorzystywana była funkcja błędu categorical_crossentropy, w przypadku Autoenkoderów model rekonstruuje dane wejściowe, więc najłatwiej wykorzystać mean square error (mse),
- w związku z powyższym również wyjście sieci się różni, nie klasyfikujemy (y_train) tylko rekonstruujemy (x_train)

https://blog.keras.io/building-autoencoders-in-keras.html

In [17]:
from tensorflow.keras.layers import UpSampling2D

class Encoder(Model):

    def __init__(self, num_classes=10):
        super(Encoder, self).__init__(name='encoder')
        self.num_classes = num_classes
        # Define your layers here.
        self.conv_1 = Conv2D(16, (3, 3), activation='relu', padding='same')
        self.pooling = MaxPooling2D((2, 2), padding='same')
        self.conv_2 = Conv2D(8, (3, 3), activation='relu', padding='same')
        self.pooling_2 = MaxPooling2D((2, 2), padding='same')
        self.conv_3 = Conv2D(8, (3, 3), activation='relu', padding='same')
        self.encoded = MaxPooling2D((2, 2), padding='same')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.conv_1(inputs)
        x = self.pooling(x)
        
        x = self.conv_2(x)
        x = self.pooling(x)
        x = self.conv_3(x)
        x = self.encoded(x)
        return x


class Decoder(Model):

    def __init__(self, num_classes=10):
        super(Decoder, self).__init__(name='decoder')
        self.conv_1 = Conv2D(8, (3, 3), activation='relu', padding='same')
        self.upSamp_1 = UpSampling2D((2, 2))
        self.conv_2 = Conv2D(8, (3, 3), activation='relu', padding='same')
        self.upSamp_2 = UpSampling2D((2, 2))
        self.conv_3 = Conv2D(16, (3, 3), activation='relu')
        self.upSamp_3 = UpSampling2D((2, 2))
        self.conv_4 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')

    def call(self, inputs):
        x = self.conv_1(inputs)
        x = self.upSamp_1(x)
        x = self.conv_2(x)
        x = self.upSamp_2(x)
        x = self.conv_3(x)
        x = self.upSamp_3(x)
        x = self.conv_4(x)
        return x



class AutoEncoder(Model):
  
    def __init__(self):
      super(AutoEncoder,self).__init__(name="autoencoder")
      self.encoder = Encoder()
      self.decoder = Decoder()

    def call(self, inputs):
      x=self.encoder(inputs)
      return self.decoder(x)


model = AutoEncoder()

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])



model.fit(x_train, x_train, batch_size=32, epochs=3,validation_data=(x_test,y_test))




Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 88s 1ms/sample - loss: 0.6608 - accuracy: 0.8086
Epoch 2/3
60000/60000 [==============================] - 88s 1ms/sample - loss: 0.4976 - accuracy: 0.8088
Epoch 3/3
60000/60000 [==============================] - 89s 1ms/sample - loss: 0.4350 - accuracy: 0.8088


## Regularyzacja
### Zadanie 5
Rozszerz model stworzony w zadaniu 1 o dwie warstwy Dropout (Dropout - https://keras.io/layers/core/):
- jedna po warstwie MaxPooling (wartość współczynnika odrzucenia 0.25)
- druga po gęstej warstwie ukrytej (Dense), wartość współczynnika odrzucenia 0.5.

  





In [38]:
class DropoutModel(Model):

    def __init__(self, num_classes=10):
        super(DropoutModel, self).__init__(name='dropout_model')
        self.num_classes = num_classes
        self.conv_1 = Conv2D(32, (3,3), (1, 1),  activation='relu')
        self.conv_2 = Conv2D(64, (3,3), (1, 1),  activation='relu')
        self.maxPooling = MaxPooling2D((2, 2))
        self.dropout_1 = Dropout(0.25)
        self.flatten = Flatten()
        self.dense =Dense(128, 'relu')
        self.dropout_2 = Dropout(0.5)
        self.dense_2 = Dense(num_classes,'softmax')

    def call(self, inputs):
        x = self.conv_1(inputs)
        x = self.conv_2(x)
        x = self.maxPooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.dropout_2(x)
        x = self.dense_2(x)
        return x

model = DropoutModel(num_classes=10)

model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



result5 = model.fit(x_train, y_train, batch_size=32, epochs=3,validation_data=(x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 144s 2ms/sample - loss: 0.2171 - accuracy: 0.9355 - val_loss: 0.0537 - val_accuracy: 0.9832
Epoch 2/3
60000/60000 [==============================] - 143s 2ms/sample - loss: 0.0780 - accuracy: 0.9776 - val_loss: 0.0368 - val_accuracy: 0.9881
Epoch 3/3
60000/60000 [==============================] - 143s 2ms/sample - loss: 0.0604 - accuracy: 0.9821 - val_loss: 0.0367 - val_accuracy: 0.9874


### Zadanie 6
Rozszerz model stworzony w zadaniu 1 o dwie warstwy Batch normalization (BatchNormalization - https://keras.io/layers/normalization/) po warstwach konwolucyjnych.

In [39]:
class BatchModel(Model):

    def __init__(self, num_classes=10):
        super(BatchModel, self).__init__(name='batch_model')
        self.num_classes = num_classes
        # Define your layers here.
        self.conv_1 = Conv2D(32, (3,3), (1, 1),  activation='relu')
        self.batch_1 = BatchNormalization()
        self.conv_2 = Conv2D(64, (3,3), (1, 1),  activation='relu')
        self.batch_2 = BatchNormalization()
        self.maxPooling = MaxPooling2D((2, 2))
        self.flatten = Flatten()
        self.dense =Dense(128, 'relu')
        self.dense_2 = Dense(num_classes,'softmax')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.conv_1(inputs)
        x = self.batch_1(x)
        x = self.conv_2(x)
        x = self.batch_2(x)
        x = self.maxPooling(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.dense_2(x)
        return x

model = BatchModel(num_classes=10)

model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



result6 = model.fit(x_train, y_train, batch_size=32, epochs=3,validation_data=(x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 173s 3ms/sample - loss: 0.2379 - accuracy: 0.9530 - val_loss: 0.2436 - val_accuracy: 0.9222
Epoch 2/3
60000/60000 [==============================] - 172s 3ms/sample - loss: 0.0625 - accuracy: 0.9856 - val_loss: 0.0723 - val_accuracy: 0.9857
Epoch 3/3
60000/60000 [==============================] - 173s 3ms/sample - loss: 0.0418 - accuracy: 0.9904 - val_loss: 0.0811 - val_accuracy: 0.9841


### Zadanie 7
Rozszerz model stworzony w zadaniu 1 o warstwy z zadań 5 i 6.

In [40]:
class FinalModel(Model):

    def __init__(self, num_classes=10):
        super(FinalModel, self).__init__(name='final_model')
        self.num_classes = num_classes
        # Define your layers here.
        self.conv_1 = Conv2D(32, (3,3), (1, 1),  activation='relu')
        self.batch_1 = BatchNormalization()
        self.conv_2 = Conv2D(64, (3,3), (1, 1),  activation='relu')
        self.batch_2 = BatchNormalization()
        self.maxPooling = MaxPooling2D((2, 2))
        self.dropout_1 = Dropout(0.25)
        self.flatten = Flatten()
        self.dense =Dense(128, 'relu')
        self.dropout_2 = Dropout(0.5)
        self.dense_2 = Dense(num_classes,'softmax')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.conv_1(inputs)
        x = self.batch_1(x)
        x = self.conv_2(x)
        x = self.batch_2(x)
        x = self.maxPooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.dropout_2(x)
        x = self.dense_2(x)
        return x

model = FinalModel(num_classes=10)

model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



result7 = model.fit(x_train, y_train, batch_size=32, epochs=3,validation_data=(x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 178s 3ms/sample - loss: 0.3573 - accuracy: 0.9139 - val_loss: 0.2878 - val_accuracy: 0.9111
Epoch 2/3
60000/60000 [==============================] - 180s 3ms/sample - loss: 0.1695 - accuracy: 0.9622 - val_loss: 0.0868 - val_accuracy: 0.9801
Epoch 3/3
60000/60000 [==============================] - 179s 3ms/sample - loss: 0.1377 - accuracy: 0.9689 - val_loss: 0.0632 - val_accuracy: 0.9842


### Zadanie 8 
Porównaj modele stworzone w zadaniach 1, 5, 6, 7. Stwórz wykresy z przebiegiem błędu funkcji celu i dokładności dla zbioru treningowego i walidacyjnego.
Cztery wykresy:
- błąd funkcji celu dla zbioru treningowego,
- błąd funkcji celu dla zbioru walidacyjnego,
- dokładność dla zbioru treningowego,
- dokładność dla zbioru walidacyjnego

Na każdym wykresie powinny być 4 przebiegi dla modeli z wszystkich zadań (1, 5, 6, 7).


In [0]:
from matplotlib import pyplot as plt

def make_plot(title, key, model1, model5, model6, model7):
  plt.plot(model1.history[key],label='Model 1')
  plt.plot(model5.history[key],label='Model 5')
  plt.plot(model6.history[key],label='Model 6')
  plt.plot(model7.history[key],label='Model 7')
  
  plt.title(title)
  plt.xlabel('epoch')
  plt.ylabel(key)
  plt.legend()
  plt.show()


make_plot("Błąd funkcji celu dla zbioru treningowego",'loss',result1,result5,result6,result7)
make_plot("Błąd funkcji celu dla zbioru walidacyjnego",'val_loss',result1,result5,result6,result7)
make_plot("Dokładność dla zbioru treningoweg",'accuracy',result1,result5,result6,result7)
make_plot("Dokładność dla zbioru walidacyjnego",'val_accuracy',result1,result5,result6,result7)